In [9]:
import torch

In [ ]:
path = 'data/mnist/'

In [ ]:
Let's download, unzip, and format the data.

In [93]:
from pathlib import Path

In [97]:
DATA_PATH = Path('data')
PATH = DATA_PATH/'mnist'

In [99]:
PATH.mkdir(parents=True, exist_ok=True)

In [100]:
URL='http://deeplearning.net/data/mnist/'
FILENAME='mnist.pkl.gz'

In [101]:
import requests

content = requests.get(URL+FILENAME).content
open(f'{PATH}/{FILENAME}', 'wb').write(content)

16168813

In [105]:
import pickle, gzip

In [106]:
((x, y), (x_valid, y_valid), _) = pickle.load(gzip.open(PATH/FILENAME, 'rb'), encoding='latin-1')

In [107]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [5]:
X = torch.tensor([[0.,0.], [0,1], [1,0], [1,1]])
y = torch.tensor([0.,1,1,0])

In [6]:
X

tensor([[ 0.,  0.],
        [ 0.,  1.],
        [ 1.,  0.],
        [ 1.,  1.]])

In [7]:
y

tensor([ 0.,  1.,  1.,  0.])

In [10]:
from torch import nn

In [12]:
torch.randn(1)

tensor([ 2.2964])

In [80]:
class LogReg(nn.Module):
    def __init__(self):
        super().__init__()
        # weights
        self.w = nn.Parameter(torch.randn(2))
        # bias
        self.b = nn.Parameter(torch.randn(1))

    def forward(self, x):
        x = x.view(x.size(0), -1)
        # Linear Layer
        x = (x @ self.w) + self.b
#         return x
        # Non-linear (sigmoid) Layer
#         return 1 / (1 + torch.exp(-x))
        return F.sigmoid(x)

In [81]:
m = LogReg()

In [82]:
preds = m(X); preds

In [40]:
def binom_loglik(input, target):
    return target*input - (1-target)*(1-input)

In [41]:
binom_loglik(preds, y)

tensor([-2.4665, -0.7460, -2.3377, -2.6172])

In [65]:
import torch.nn.functional as F

In [68]:
loss_fn = F.binary_cross_entropy

In [84]:
loss_fn(preds, y)

tensor(0.7651)

In [85]:
lr = 1.

In [88]:
for epoch in range(30):
    pred = m(X)
    loss = loss_fn(pred, y)

    # Backward pass: compute gradient of the loss with respect to model parameters
    loss.backward()
    m.w.data -= m.w.grad.data * lr
    m.b.data -= m.b.grad.data * lr

    m.zero_grad()

In [90]:
preds = m(X); preds

tensor([ 0.5811,  0.5267,  0.4991,  0.4442])

In [91]:
loss_fn(preds, y)

tensor(0.6984)

- Validation
- CUDA
- Conv

In [26]:
def get_weights(*dims): return nn.Parameter(torch.randn(dims)/dims[0])
def softmax(x): return torch.exp(x)/(torch.exp(x).sum(dim=-1)[:,None])